## 2.3 Course Information Modeling: predicting enrollment

### Import libraries

In [2]:
# the basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_colwidth = 350

In [3]:
#import preprocessing and metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [4]:
# import modeling
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [53]:
import warnings
warnings.filterwarnings('ignore')

### Import Data and quick cleaning

In [5]:
path = '../data/ds_course_modeling.csv'
course = pd.read_csv(path)

# quick pre-processing, dropping columns and remove rows with null values
course.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
# check shape
course.shape

(348, 30)

In [7]:
# check null values
course.isnull().sum()

course_href                 0
course_name                 0
partner_title               0
stars                       0
recent_views                0
num_ratings                 0
num_reviews                 0
description                 0
length                      0
100%_online                 0
shareable_certificate       0
flexible_deadlines          0
english                     0
intermediate_level          0
beginner_level              0
spanish                     0
chinese_(traditional)       0
arabic                      0
portuguese_(brazilian)      0
russian                     0
advanced_level              0
chinese_(simplified)        0
french                      0
japanese                    0
specialization              0
outcome_career_benefit    153
outcome_pay_increase      225
outcome_new_career        180
he_partner                  0
enrollment                  0
dtype: int64

In [8]:
# fill all null values with 'median'
course.fillna('median', inplace=True)

In [9]:
course.head(1)

,course_href,course_name,partner_title,stars,recent_views,num_ratings,num_reviews,description,length,100%_online,...,advanced_level,chinese_(simplified),french,japanese,specialization,outcome_career_benefit,outcome_pay_increase,outcome_new_career,he_partner,enrollment
0,/learn/exploratory-data-analysis,Exploratory Data Analysis,Johns Hopkins University,4.7,108049,5836,845,This course covers the essential exploratory techniques for summarizing data. These techniques are typically applied before formal modeling commences and can help inform the development of more complex statistical models. Exploratory techniques are also important for eliminating or sharpening potential hypotheses about the world that can be add...,55,1,...,0,0,0,0,0,38.0,15.0,38.0,1,157581


---
## Model Selection

### Baseline Model: mean enrollment (y)

In [28]:
y = course['enrollment']
# null prediction
null_predictions = pd.Series(data=y.mean(), index=y.index)
# null residuals
null_residuals = y - null_predictions

In [29]:
# null RMSE
metrics.mean_squared_error(y, null_predictions, squared=False)

235563.11860296366

### 1. Modeling: Using all numeric features to predict enrollment.
#### Set up X, y, train-test-split

In [30]:
# get all numeric features
numeric_features = course._get_numeric_data().columns

In [31]:
X = course[numeric_features].drop(columns=['enrollment'])
y = course['enrollment']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Setting up the models in a pipeline and evaluating performance using R2 and RMSE

In [33]:
# Set up a list to contain pipelines
pipelines = []

# Set up tuples for all the models we want to test
models = [('lr', LinearRegression()),
         ('knn', KNeighborsRegressor()),
         ('dtr', DecisionTreeRegressor()),
         ('rfr', RandomForestRegressor()),
         ('ada', AdaBoostRegressor()),
         ('svr', SVR()),
         ('xgb', XGBRegressor())]

# Set up pipelines for all the models alone and with StandardScaler
for model in models:
    pipelines.append(Pipeline([('ss', StandardScaler()),
                              model]))

In [42]:
# this function takes a list of pipelines and return their R2 and RMSE scores 
def pipe_evaluations(pipelines):
    evaluations = []

# Loop through and fit all the pipelines
    for pipe in pipelines:
        pipe.fit(X_train, y_train)

        evaluations.append({
            'Pipeline': [step[0] for step in pipe.steps],
            'R2 (train)': pipe.score(X_train, y_train),
            'R2 (test)': pipe.score(X_test, y_test),
            'RMSE (train)': metrics.mean_squared_error(y_true =y_train, y_pred = pipe.predict(X_train),
                                              squared = False),
            'RMSE (test)': metrics.mean_squared_error(y_true =y_test, y_pred = pipe.predict(X_test),
                                              squared = False)
        })

    # add evaluation metrics to a dataframe
    evaluations_df = pd.DataFrame(evaluations)
    return evaluations_df

In [44]:
# evaluation the pipelines
pipe_evaluations(pipelines)

,Pipeline,R2 (train),R2 (test),RMSE (train),RMSE (test)
0,"[ss, lr]",0.950881,0.580621,57866.614044,44385.917368
1,"[ss, knn]",0.559316,0.579777,173327.493808,44430.557450
2,"[ss, dtr]",1.000000,0.821153,0.000000,28985.633775
3,"[ss, rfr]",0.872073,0.623064,93386.591764,42079.997481
4,"[ss, ada]",0.980803,0.766812,36175.727129,33097.477841
5,"[ss, svr]",-0.040256,-0.182886,266301.826967,74544.058599
6,"[ss, xgb]",1.000000,0.862996,153.023514,25369.305579


#### Shortlisting models based on performance

**Takeaways:** based on the test model performance (R2 scores and RMSE), the two models were selected for GridSearch.
- **Decision Tree**
- **XGBoost**


**In addition,** for interpretability, I will also continue exploring the **Linear Regression** model and will apply polynominal features and regularization as next steps.

### 2. Excluding `100%_online`, `flexible_deadlines` and all the 'language labels from X
#### Set up X, y

In [244]:
X = course[numeric_features].drop(columns=['spanish','chinese_(traditional)', 'arabic', 'portuguese_(brazilian)', 
                                           'russian', 'advanced_level', 'chinese_(simplified)', 'french', 
                                           'japanese', '100%_online', 'flexible_deadlines', 'enrollment'])
y = course['enrollment']

In [245]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Setting up the models in a pipeline and evaluating performance using R2 and RMSE

In [246]:
# Set up a list to contain pipelines
pipelines = []

# Set up tuples for all the models we want to test
models = [('lr', LinearRegression()),
         ('knn', KNeighborsRegressor()),
         ('dtr', DecisionTreeRegressor()),
         ('rfr', RandomForestRegressor()),
         ('ada', AdaBoostRegressor()),
         ('svr', SVR()),
         ('xgb', XGBRegressor())]

# Set up pipelines for all the models alone and with StandardScaler
for model in models:
    pipelines.append(Pipeline([('ss', StandardScaler()),
                              model]))

In [247]:
# evaluation the pipelines
pipe_evaluations(pipelines)

,Pipeline,R2 (train),R2 (test),RMSE (train),RMSE (test)
0,"[ss, lr]",0.950881,0.580621,57866.614044,44385.917368
1,"[ss, knn]",0.559316,0.579777,173327.493808,44430.557450
2,"[ss, dtr]",1.000000,0.837661,0.000000,27615.473721
3,"[ss, rfr]",0.920195,0.694450,73759.709359,37886.354880
4,"[ss, ada]",0.981504,0.702076,35509.711634,37410.585537
5,"[ss, svr]",-0.040256,-0.182886,266301.826967,74544.058599
6,"[ss, xgb]",1.000000,0.862996,153.023514,25369.305579


**Takeaways:** based on the test model performance (R2 scores and RMSE), the two models were selected for GridSearch.
- **Decision Tree**
- **XGBoost**


For interpretability, I will also continue exploring the **Linear Regression** model and will apply polynominal features and regularization as next steps. **In addition,** Neural Networks are also explored as a benchmark to compare model performance.

---
## Tune shortlisted models, using the X excluding `100%_online`, `flexible_deadlines` and all the 'language labels  
Shortlisted models:
- **Decision Tree** (tuning using GridSearch)
- **XGBoost** (tuning using GridSearch??)
- **Linear Regression** (tuning using polynormial features and regularization)

In [51]:
from sklearn.model_selection import GridSearchCV

In [248]:
X = course[numeric_features].drop(columns=['spanish','chinese_(traditional)', 'arabic', 'portuguese_(brazilian)', 
                                           'russian', 'advanced_level', 'chinese_(simplified)', 'french', 
                                           'japanese', '100%_online', 'flexible_deadlines', 'enrollment'])
y = course['enrollment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### GridSearch - Decision Tree Regressor

In [249]:
# without tuning
pipe = Pipeline([('ss', StandardScaler()),
                  ('dtr', DecisionTreeRegressor())])


pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(1.0, 0.8198534222259319)

In [250]:
# set up the pipe
pipe = Pipeline([('ss', StandardScaler()),
                  ('dtr', DecisionTreeRegressor())])

# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [{'dtr__max_depth':[11,12,13,None],
               'dtr__min_samples_split':[4,6,8],
               'dtr__min_samples_leaf': [1,2,3],
               'dtr__max_leaf_nodes': [10,11,12,None]}]

# create a gridsearch of the pipeline, and fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=5) 
best_model = gridsearch.fit(X_train,y_train)

# print metrics
print(best_model.best_estimator_)
print('')
print(best_model.score(X_train, y_train), best_model.score(X_test, y_test))

Pipeline(steps=[('ss', StandardScaler()),
                ('dtr',
                 DecisionTreeRegressor(min_samples_leaf=2,
                                       min_samples_split=8))])

0.7385200136660097 0.8383064760082237


**Key takeways:** R-square score improves from **0.820 to 0.838** with GridSearching the Decision Tree Model.

### GridSearch - XGBoost

**Performance without GridSearch:** Before tuning XGBoost model, take a look at the performance of the default setting. There is overfitting in the model and I will try to tune it by adding some bias.

In [251]:
# without tuning
pipe = Pipeline([('ss', StandardScaler()),
                  ('xgb', XGBRegressor())])


pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9999996565146364, 0.8629958220673176)

In [252]:
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [{'xgb__max_depth':[4,5],
               'xgb__min_child_weight':[1,2],
               'xgb__n_estimators': [100,200],
               'xgb__learning_rate': [0.2,0.3,0.4]}]

# create a gridsearch of the pipeline, and fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=5) 
best_model = gridsearch.fit(X_train,y_train)

print(best_model.best_estimator_)
print('')
print(best_model.score(X_train, y_train), best_model.score(X_test, y_test))

Pipeline(steps=[('ss', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.2,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

0.9998210419423381 0.884980902946525


**Key takeways:** R-square score improves from **0.863 to 0.885** with GridSearching the XGBoost Model.

### Linear Regression and Regularization

In [253]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV

In [255]:
# Recall:
X = course[numeric_features].drop(columns=['spanish','chinese_(traditional)', 'arabic', 'portuguese_(brazilian)', 
                                           'russian', 'advanced_level', 'chinese_(simplified)', 'french', 
                                           'japanese', '100%_online', 'flexible_deadlines', 'enrollment'])
y = course['enrollment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Linear Regression without regularization

In [256]:
pipe = Pipeline([('ss', StandardScaler()),
                 ('lr', LinearRegression())])

pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9508811676656268, 0.5806206401638387)

#### LASSO with PolynomialFeatures

In [257]:
# set up the pipe
pipe = Pipeline([('poly', PolynomialFeatures()),
                  ('ss', StandardScaler()),
                  ('lasso', Lasso())])

pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9932006717582604, 0.8702241963094685)

In [259]:
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [{'poly__degree':[2,3],
               'poly__interaction_only':[True, False],
               'lasso__alpha': [1, 10, 100]}]

# create a gridsearch of the pipeline, and fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=5) 
best_model = gridsearch.fit(X_train,y_train)

print(best_model.best_estimator_)
print('')
print(best_model.score(X_train, y_train), best_model.score(X_test, y_test))

Pipeline(steps=[('poly', PolynomialFeatures(degree=3, interaction_only=True)),
                ('ss', StandardScaler()), ('lasso', Lasso(alpha=10))])

0.9950073365792866 0.882777862703681


**Key takeways:** R-square score improves from **0.581 to 0.883** with LASSO Regularization.

#### Ridge with PolynomialFeatures"

In [260]:
pipe = Pipeline([('poly', PolynomialFeatures()),
                  ('ss', StandardScaler()),
                  ('ridge', Ridge())])

pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9924096696864031, 0.8622941131736972)

In [261]:
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [{'poly__degree':[2,3],
               'poly__interaction_only':[True, False],
               'ridge__alpha': [0.01,0.1,1]}]

# create a gridsearch of the pipeline, and fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=5) 
best_model = gridsearch.fit(X_train,y_train)

print(best_model.best_estimator_)
print('')
print(best_model.score(X_train, y_train), best_model.score(X_test, y_test))

Pipeline(steps=[('poly', PolynomialFeatures(degree=3, interaction_only=True)),
                ('ss', StandardScaler()), ('ridge', Ridge(alpha=1))])

0.9941874207858192 0.8683042328727619


**Key takeways:** R-square score improves from **0.581 to 0.868** with Ridge Regularization.

#### Using LASSO for feature selection and intepretation

In [262]:
from sklearn.feature_selection import SelectFromModel
# reference:
# https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499

In [285]:
poly = PolynomialFeatures(degree=3, interaction_only=False)
X_train_overfit = poly.fit_transform(X_train)
X_test_overfit = poly.transform(X_test)
sc = StandardScaler()
X_train_overfit_sc = sc.fit_transform(X_train_overfit)
X_test_overfit_sc = sc.transform(X_test_overfit)
lasso_lr = Lasso(alpha=7500)
lasso_lr.fit(X_train_overfit_sc, y_train)
lasso_lr.score(X_train_overfit_sc, y_train), lasso_lr.score(X_test_overfit_sc, y_test)

(0.9860108160495665, 0.6999168399560736)

In [286]:
# can also use SelectFromModel 

# from sklearn.feature_selection import SelectFromModel
# reference:
# https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499

# sel_ = SelectFromModel(Lasso(alpha=3000))
# sel_.fit(X_train_overfit_sc, y_train)

In [291]:
# put the selected features into a DataFrame
selections = pd.DataFrame(data=lasso_lr.coef_, index=poly.get_feature_names(X_train.columns), columns=['model_coefficient'])

# list the features of the non-zero coefficients 
selections[selections['model_coefficient']!=0]

,model_coefficient
num_ratings,26699.244012
num_reviews,50124.894204
num_ratings length,67209.707451
num_ratings shareable_certificate,3684.908363
num_ratings he_partner,106712.282179
num_reviews shareable_certificate,1536.819649
length shareable_certificate,112.550500
num_ratings shareable_certificate he_partner,986.866842
num_ratings he_partner^2,897.318160
num_reviews^2 beginner_level,-4996.774358


In [290]:
# get the feature names of the non-zero features
selections[selections['model_coefficient']!=0].index

Index(['num_ratings', 'num_reviews', 'num_ratings length',
       'num_ratings shareable_certificate', 'num_ratings he_partner',
       'num_reviews shareable_certificate', 'length shareable_certificate',
       'num_ratings shareable_certificate he_partner',
       'num_ratings he_partner^2', 'num_reviews^2 beginner_level',
       'num_reviews shareable_certificate^2', 'length^2 he_partner',
       'length shareable_certificate^2'],
      dtype='object')

**Key Takeways**:  
The most important features are: `'num_ratings', 'num_reviews', 'num_ratings length', 'num_ratings shareable_certificate', 'num_ratings he_partner', 'num_reviews shareable_certificate', 'length shareable_certificate', 'num_ratings shareable_certificate he_partner', 'num_ratings he_partner^2', 'num_reviews^2 beginner_level', 'num_reviews shareable_certificate^2', 'length^2 he_partner', 'length shareable_certificate^2'`   
All these features and their poly/interaction terms are positively related to enrollment except for `num_reviews^2 beginner_level`. 

In [292]:
# end of notebook